In [10]:
import pandas as pd
from datetime import date
import json
import numpy
import requests
import datetime as dt

# Municipios

In [107]:
municipios_distritos=pd.read_csv('municipios_distritos.csv')

In [114]:
def get_municipios(date):
    
    data= requests.get("https://services1.arcgis.com/7bh7e06dYKE4ZqXL/arcgis/rest/services/Municipios_y_Distritos_Automatizado/FeatureServer/0/query?where=OBJECTID+%3E+1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=false&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=none&f=pjson&token=")
    municipios= data.json()
    municipios=municipios['features']
    
    m = []
    for municipio in municipios:
        m.append(municipio['attributes'])

    municipios= pd.DataFrame(m)
    municipios = municipios[['municipio_distrito','fecha_informe', 'tasa_inc_acum_ult_14dias','tasa_inc_acum_total',
                         'casos_confirmados_totales']]
    
    municipios['fecha_informe']= date
    municipios['fecha_informe']= pd.to_datetime(municipios['fecha_informe'])
    municipios['fecha_informe'] = municipios['fecha_informe'].dt.date
    
    
    return municipios

In [115]:
last_update= get_municipios(date='2020.12.01')

In [116]:
municipios_distritos= municipios_distritos.append(last_update)

In [119]:
municipios_distritos.to_csv("municipios_distritos.csv", index=False, encoding='UTF-8')  

## Zonas básicas

In [120]:
areas_salud=pd.read_csv('areas_salud.csv')

In [131]:
def get_zones(date):
    
    data= requests.get("https://services1.arcgis.com/7bh7e06dYKE4ZqXL/arcgis/rest/services/Zonas_Basicas_de_Salud_Automatizado/FeatureServer/0/query?where=OBJECTID+%3E+1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=false&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token=")
    zones= data.json()
    zones=zones['features']
    
    z = []
    for zone in zones:
        z.append(zone['attributes'])

    zones= pd.DataFrame(z)
    zones = zones[['casos_conf_ult_14dias','casos_confirmados_totales', 'zona_basic', 'fecha_informe',
                'tasa_inc_acum_ult_14dias', 'tasa_inc_acum_total']]
    
    zones['fecha_informe']= date
    zones['fecha_informe']= pd.to_datetime(zones['fecha_informe'])
    zones['fecha_informe'] = zones['fecha_informe'].dt.date
    
    
    return zones

In [132]:
last_update=get_zones(date='2020.12.01')

In [133]:
areas_salud = areas_salud.append(last_update)

In [83]:
areas_salud.to_csv("areas_salud.csv", index=False, encoding='UTF-8')  

# Teoria

Lo primero, coger el ID del mapa

In [122]:
webmap = gis.content.get('7db220dc2e0a40b4a928df661a89762e')
webmap

<Item title:"Acceso Mapa covid19_tia_zonas_basicas_salud" type:Web Mapping Application owner:dgsp.Sanidad>

### Lo segundo, buscar los items del mapa y fijarse en la clave del valor "webmap"

In [123]:
r = requests.get('https://www.arcgis.com/sharing/rest/content/items/7db220dc2e0a40b4a928df661a89762e/data')

In [124]:
r.text

'{"source":"95ccb288429b4127a07c0d8a8a65b4d5","folderId":null,"values":{"enableBasemapToggle":true,"enableScalebar":true,"twitterSearch":"","youtubeVisible":false,"locationSearch":true,"flickrVisible":false,"enableLocateButton":true,"enableAboutPanel":false,"enableBookmarks":false,"enableHomeButton":true,"defaultPanel":"legend","openOverviewMap":false,"title":"COVID-19 Situación epidemiológica por zonas de salud 24/11/2020","flickrTime":"all_time","twitterVisible":true,"defaultBasemap":"gray","enableViewsCount":true,"enablePrintButton":false,"enableLegendPanel":true,"dialogModalContent":"","flickrSearch":"","youtubeSearch":"","swipeInvertPlacement":false,"enableFlickr":false,"dialogModalTitle":"","enableSummaryInfo":false,"enableOverviewMap":true,"webcamsVisible":false,"enableTwitter":false,"enableInstagram":true,"enableShareDialog":false,"enableWebcams":false,"enableYouTube":false,"youtubeTime":"all_time","enableDialogModal":false,"swipeType":"vertical","hideNotesLayerPopups":true,"in

### Luego, fijarse en la url que apaerece ene l operationalLayers

In [13]:
i = requests.get('https://www.arcgis.com/sharing/rest/content/items/d5e7be360ec2404eaed6a16a9801ed0a/data')
i.text


'{"operationalLayers":[{"id":"Zonas_Basicas_de_Salud_Automatizado_6172_854","layerType":"ArcGISFeatureLayer","url":"https://services1.arcgis.com/7bh7e06dYKE4ZqXL/arcgis/rest/services/Zonas_Basicas_de_Salud_Automatizado/FeatureServer/0","visibility":true,"opacity":0.52,"title":"Zonas Basicas de Salud","itemId":"e3c41f4cb7324b56a6e8e8eb1a5238df"}],"baseMap":{"baseMapLayers":[{"id":"World_Light_Gray_Base_1486","layerType":"ArcGISTiledMapServiceLayer","url":"https://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer","visibility":true,"opacity":1,"title":"Base Lona gris claro mundial"}],"title":"Lona gris claro"},"spatialReference":{"wkid":102100,"latestWkid":3857},"authoringApp":"WebMapViewer","authoringAppVersion":"8.3","version":"2.18"}'

Acceder a la web y hacer la query con los parámetros:
* Field	Value
* Where:	OBJECTID > 0
* Out Fields:	*
* Return Geometry:	False

In [19]:
munici= requests.get('https://www.arcgis.com/sharing/rest/content/items/cdfb61b3eb3a49c2b990b4fdb41dfcfe/data')
munici.text

'{"source": "95ccb288429b4127a07c0d8a8a65b4d5", "folderId": null, "values": {"enableBasemapToggle": true, "enableScalebar": true, "twitterSearch": "", "youtubeVisible": false, "locationSearch": true, "flickrVisible": false, "enableLocateButton": true, "enableAboutPanel": false, "enableBookmarks": false, "enableHomeButton": true, "defaultPanel": "legend", "openOverviewMap": false, "title": "COVID-19 Situaci\\u00f3n epidemiol\\u00f3gica por municipios 24/11/2020", "flickrTime": "all_time", "twitterVisible": false, "defaultBasemap": "gray", "enableViewsCount": true, "enablePrintButton": false, "enableLegendPanel": true, "dialogModalContent": "", "flickrSearch": "", "youtubeSearch": "", "swipeInvertPlacement": false, "enableFlickr": false, "dialogModalTitle": "", "enableSummaryInfo": false, "enableOverviewMap": true, "webcamsVisible": false, "enableTwitter": false, "enableInstagram": true, "enableShareDialog": false, "enableWebcams": false, "enableYouTube": false, "youtubeTime": "all_time"

In [20]:
i = requests.get('https://www.arcgis.com/sharing/rest/content/items/7acfe1f4123649c8b870deff65d0f782/data')
i.text


'{"operationalLayers":[{"id":"Municipios_y_Distritos_Automatizado_2061_7543","layerType":"ArcGISFeatureLayer","url":"https://services1.arcgis.com/7bh7e06dYKE4ZqXL/arcgis/rest/services/Municipios_y_Distritos_Automatizado/FeatureServer/0","visibility":true,"opacity":0.5,"title":"Municipios y Distritos","itemId":"bee8fd8a853e441992d3ff19619ff9bc"}],"baseMap":{"baseMapLayers":[{"id":"World_Light_Gray_Base_1486","layerType":"ArcGISTiledMapServiceLayer","url":"https://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer","visibility":true,"opacity":1,"title":"Base Lona gris claro mundial","minScale":5.91657527591555E8,"maxScale":70.5310735}],"title":"Lona gris claro"},"spatialReference":{"wkid":102100,"latestWkid":3857},"authoringApp":"WebMapViewer","authoringAppVersion":"8.3","version":"2.18","applicationProperties":{"viewing":{"search":{"enabled":true,"disablePlaceFinder":true,"hintText":"Lugar o dirección","layers":[{"id":"Municipios_y_Distritos_Automatiza

In [134]:
x= requests.get("https://services1.arcgis.com/7bh7e06dYKE4ZqXL/arcgis/rest/services/Municipios_y_Distritos_Automatizado/FeatureServer/0/query?where=OBJECTID+%3E+1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=false&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=false&quantizationParameters=&sqlFormat=none&f=pjson&token=")
municipios= x.json()

In [135]:
municipios.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [136]:
municipios=municipios['features']

In [137]:
m = []
for muni in municipios:
    m.append(muni['attributes'])


In [138]:
municipios= pd.DataFrame(m)

In [139]:
municipios = municipios[['municipio_distrito','fecha_informe', 'tasa_inc_acum_ult_14dias','tasa_inc_acum_total',
                         'casos_confirmados_totales']]

In [171]:
municipios['fecha_informe']= '2020-11-24'	

In [172]:
municipios['fecha_informe']= pd.to_datetime(municipios['fecha_informe'])

In [140]:
municipios.to_csv("municipios_distritos.csv", index=False)  

In [180]:
data= requests.get("https://services1.arcgis.com/7bh7e06dYKE4ZqXL/arcgis/rest/services/Zonas_Basicas_de_Salud_Automatizado/FeatureServer/0/query?where=OBJECTID+%3E+1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=false&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token=")
zones= data.json()

In [182]:
zones= pd.DataFrame(z)

In [183]:
zones = distri[['casos_conf_ult_14dias','casos_confirmados_totales', 'zona_basic', 'fecha_informe',
                'tasa_inc_acum_ult_14dias', 'tasa_inc_acum_total']]

In [185]:
zones['fecha_informe']= pd.to_datetime(zones['fecha_informe'])

In [187]:
zones.to_csv("areas_salud.csv", index=False, encoding='UTF-8')  